In [7]:
import requests 
import json
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke
from forex_python.converter import CurrencyRates

pd.set_option('display.max_rows', None)

# Задание 1

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.


In [2]:
# Mongo DB, создание коллекции

client = MongoClient('127.0.0.1', 27017)
db = client['vac_database']
vac_collection = db.vacancies_hh
vac_collection

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'vac_database'), 'vacancies_hh')

In [3]:
# Функция добавляет Вакансии в созданную коллекцию
def insert_vacancies(collection, add_vacancy):
    try:
        collection.insert_one(add_vacancy)
        print(f"Добавление вакансии {add_vacancy['vacancy']}")
    except dke:
        print(f"Вакансия по ссылке {add_vacancy['_id']} уже существует в базе")

In [4]:
# Введите интересующую вакансию, регион и кол-во страниц
vacancy_name = 'game analyst'
vacancy_region = 1
number_of_pages = 5

In [5]:
url = 'https://hh.ru'
params = {'clusters': 'true',
          'area': vacancy_region,
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': None,
          'text': vacancy_name,
          'page': 0}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

vacancies_list = []

while params['page'] < number_of_pages:
    response = requests.get(url + '/search/vacancy', params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})

    if response.ok and vacancies:
        
        for vacancy in vacancies:
            vacancy_data = {}
            info = vacancy.find('a', {'class': 'bloko-link'})
            name = info.text
            company = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
            link = info['href']
            site = url
            
            try:
                salary = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text
                i = salary.split()
                if 'от' in salary:
                    min_salary = int(i[1] + i[2])
                    max_salary = None
                    fix_salary = None
                    currency = i[3]
                elif 'до' in salary:
                    min_salary = None
                    max_salary = int(i[1] + i[2])
                    fix_salary = None
                    currency = i[3]
                elif ' - ' in salary:
                    min_salary = int(i[0] + i[1])
                    max_salary = int(i[3] + i[4])
                    fix_salary = None
                    currency = i[5]
                else:
                    min_salary = None
                    max_salary = None
                    fix_salary = int(i[0] + i[1])
                    currency = i[2]
            except:
                #salary = None
                min_salary = None
                max_salary = None
                fix_salary = None
                currency = None
            
            # Ссылка на вакансию в качестве уникального id
            vacancy_data['_id'] = link
            vacancy_data['vacancy'] = name
            vacancy_data['company'] = company
            vacancy_data['link'] = link
            vacancy_data['source'] = site
            vacancy_data['min_salary'] = min_salary
            vacancy_data['max_salary'] = max_salary
            vacancy_data['fix_salary'] = fix_salary
            vacancy_data['currency'] = currency
            
            vacancies_list.append(vacancy_data)
            
            # Добавления вакансии в базу
            insert_vacancies(vac_collection, vacancy_data)
            
        print(f"Обработана {params['page']} страница")
        params['page'] += 1
    else:
        break
        
#result = pd.DataFrame(vacancies_list)
#pprint(vacancies_list)

Вакансия по ссылке https://hh.ru/vacancy/48809730?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/49220216?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/48725827?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/48357502?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/47608292?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/49209976?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/49030653?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/46256769?from=vacancy_search_list&query=game%20analyst уже существует в базе
Вакансия по ссылке https://hh.ru/vacancy/48332891?from=v

In [6]:
result = pd.DataFrame(vacancies_list)
result['currency'].unique()

array([None, 'руб.', '–', 'USD'], dtype=object)

# Задание 2

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). 

In [25]:
#pip install forex-python
# Библиотека для конвертации валюты
# Пара USD к RUB

c = CurrencyRates()
c_usd_to_rub = c.get_rate('USD', 'RUB')
c_usd_to_rub

71.71929673360337

In [26]:
# Функция поиска вакансий с заработной платой выше введеной пользователем

def search_gt_salary(collection):
    count_num = 1
    try:
        user_salary_input = int(input("Введите сумму ЗП в USD: "))
        for i in collection.find({'currency': 'USD', 
                                  '$or': [{'max_salary':{'$gt':user_salary_input}},
                                         {'min_salary':{'$gt':user_salary_input}},
                                         {'fix_salary':{'$gt':user_salary_input}}]},
                                {'_id':0}):
            pprint(f'{count_num} {i}')
            count_num += 1
        for i in collection.find({'currency': 'руб.', 
                                  '$or': [{'max_salary':{'$gt':user_salary_input / c_usd_to_rub}},
                                         {'min_salary':{'$gt':user_salary_input / c_usd_to_rub}},
                                         {'fix_salary':{'$gt':user_salary_input / c_usd_to_rub}}]},
                                {'_id':0}):
            pprint(f'{count_num} {i}')
            count_num += 1
    except ValueError:
        print('Сумма должна быть указано цифрой')

In [27]:
search_gt_salary(vac_collection)

Введите сумму ЗП в USD: 1500
("1 {'vacancy': 'Game Producer (Hyper Casual Games)', 'company': 'Храмцова "
 "Полина', 'link': "
 "'https://hh.ru/vacancy/48357502?from=vacancy_search_list&query=game%20analyst', "
 "'source': 'https://hh.ru', 'min_salary': 250000, 'max_salary': None, "
 "'fix_salary': None, 'currency': 'руб.'}")
("2 {'vacancy': 'Lead UA Manager (ведущий трафик-менеджер)', 'company': "
 "'Kometa.Games', 'link': "
 "'https://hh.ru/vacancy/49265007?from=vacancy_search_list&query=game%20analyst', "
 "'source': 'https://hh.ru', 'min_salary': None, 'max_salary': 280000, "
 "'fix_salary': None, 'currency': 'руб.'}")
("3 {'vacancy': 'Арбитражник / Трафик-менеджер ( Digital-marketing )', "
 "'company': 'ООО\\xa0Кадровое агентство Даподбор', 'link': "
 "'https://hh.ru/vacancy/48409844?from=vacancy_search_list&query=game%20analyst', "
 "'source': 'https://hh.ru', 'min_salary': 170000, 'max_salary': None, "
 "'fix_salary': None, 'currency': 'руб.'}")
